# input the right format from mongo

In [43]:
import pymongo
from pymongo import MongoClient
from gensim.models import word2vec
client = pymongo.MongoClient('localhost',27017)
db = client.sinicut_sw
corpus = []
for dic in db.plaintiff_sinica_sw.find({},{'_id':0}):
    article = []
    for ele in dic:
        article.append(ele)
    corpus.append(article)
model = word2vec.Word2Vec(corpus,min_count=10 , size = 100, workers=4) #, window=10,, sample=1e-5
model.save('sinicut_all_size100')



# input model formation by location and annual

In [1]:
import pymongo
from pymongo import MongoClient
from gensim.models import word2vec
client = pymongo.MongoClient('localhost',27017)
db = client.test
corpus = []
localzip =[100,111,220,260,200,320,300,350,400,500,540,700,800,970,630,602,950,900,880,890,209,850]
for location in localzip:
    #mergedic =MyDict(dicc)
    for dic in db.plaintiff_sinica_sw.find({'_id':{'$regex':'^%d_'%(location)}},{'_id':0},no_cursor_timeout=True):
        article = []
        for ele in dic:
            if ele != '_id':
                article.append(ele)
        corpus.append(article)
    model = word2vec.Word2Vec(corpus,min_count=20 , size = 200, workers=4)
    model.save('sinicut_location_%d'%(location))

In [58]:
import pymongo
from pymongo import MongoClient
from gensim.models import word2vec
client = pymongo.MongoClient('localhost',27017)
db = client.test
corpus = []
annual = [89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104]

for ayear in annual:
    #mergedic =MyDict(dicc)
    for dic in db.plaintiff_sinica_sw.find({'_id':{'$regex':'%d(\d{4})$'%(ayear)}},{'_id':0},no_cursor_timeout=True):
        article = []
        for ele in dic:
            if ele != '_id':
                article.append(ele)
        corpus.append(article)
    model = word2vec.Word2Vec(corpus,min_count=20 , size = 200, workers=4)
    model.save('sinicut_annual_%d'%(ayear))

# Training the corpus

In [27]:
#encoding=utf-8
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import word2vec

model = word2vec.Word2Vec(corpus,min_count=20 , size = 200, workers=4)
#model.train(corpus)
model.save('sinicut_all_size800')

# calculate the similarity

In [8]:
import glob,codecs

worddic = {}
for path in glob.glob('./cutTerms/sinicutonce_location2/*txt'):
    f = codecs.open(path,'r','utf-8')
    txtlist = f.readlines()
    for txt in txtlist:
        word = txt.split(',')[0]
        if word not in worddic:
            worddic[word]=1
        else:
            worddic[word]+=1
print len(worddic)
#for ele in worddic:
#    print ele,worddic[ele]
# 獨缺「居留証」


238


In [5]:
import codecs,glob

worddic = {}
for filetxt in glob.glob('./cutTerms/tfidf/*txt'):
    f = codecs.open(filetxt,'r','utf-8')
    wlist = f.readlines()
    for word in wlist:
        term = word.split(',')[1].strip()
        if term not in worddic:
            worddic[term]=1
        else:
            worddic[term]+=1

In [6]:
#encoding=utf-8
import codecs
import gensim
from gensim.models import word2vec
model = gensim.models.Word2Vec.load('word2vec/sinicut_all_size100')
#model = gensim.models.Word2Vec('sinicut_once2')
f = codecs.open('./cutTerms/word2vec_output2.txt','a+','utf-8')
for word in worddic:
    count =1
    try:
        for w in model.most_similar(word):
            intxt = word+';'+str(count).decode('utf-8')+','+w[0]+';'+'\n'
            count+=1
            f.write(intxt)
    except:
        print word
        pass
f.close()

#calculate similarity
#for w in model.most_similar('kimono'):
#    print w[0].encode('utf-8'), w[1]
#sm = model.most_similar([u'毆打'])
#for ele in sm:
#    print ele[0].encode('utf-8'),ele[1]

In [38]:
f.close()


# 檢驗兩個共同出現的案件數

In [2]:
import pymongo
from pymongo import MongoClient
client = pymongo.MongoClient('localhost',27017)
db = client.sinicut_sw
check = False
check2 = False
count = 0
for dic in db.plaintiff_sinica_sw.find({},{'_id':0}):
    for ele in dic:
        if ele == u'外遇':
            check = True
        if ele == u'婚外情':
            check2 = True
    if check & check2:
        count+=1
    check = False
    check2 = False
print count

305
